In [1]:
##필요한 모듈 import
import math
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt  
okt=Okt()  
   
    
    
##필요한 함수들
##명사, 알파벳, 숫자만 추출하는 함수
def tokenizer(raw, pos=["Noun","Alpha","Number"], stopword=[]): #현재는 "Verb" 제외
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅋ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]


##코사인 유사도를 계산하는 함수
def cos_similarity(v1, v2):
    dot_product = np.dot(v1, v2)
    l2_norm = (np.sqrt(sum(np.square(v1))) * np.sqrt(sum(np.square(v2))))
    similarity = dot_product / l2_norm     
    
    return similarity




In [44]:
##data frame을 이용해서 json 파일을 읽어온 후 dflist에 기사 100개만 넣는다.
df = pd.read_json('./re1996_03.json')
dfurl = []
dflist = []
dfcompany = []
dfperiod = []
dftitle = []

for i in range(len(df)):
    dflist.append(df['doc'][i])
    dfurl.append(df['url'][i])
    dfcompany.append(df['company'][i])
    dfperiod.append(df['period'][i])
    dftitle.append(df['title'][i])

print(len(dflist))    

1768


In [3]:

###======================tf-idf 단계===========================
##tf-idf 모듈을 사용한다.
vectorizer = TfidfVectorizer(
    tokenizer=tokenizer, min_df=2
)




##기사마다 뽑은 feature들을 숫자로 만든다.
tfidf_data = vectorizer.fit_transform(dflist)   
# print("data::\n", tfidf_data)
# print()

 
    
    
##feature 들의 목록을 출력한다. (기사에서 전처리 후 뽑힌 단어들) 
word_token = vectorizer.get_feature_names()
# print("::: word_token", ":" * 100, "\n")
# print(word_token)
# print()
 

    
##tf-idf 값을 계산한 벡터를 출력한다.
print("::: tfidf with vector", ":" * 100)
print(tfidf_data.toarray())
print()




::: tfidf with vector ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]



In [4]:
###======================kmeans 단계===========================
##kmeans 군집화를 실행한다. 
k_number = int((math.sqrt(len(dflist)/2))/2)

X = np.array(tfidf_data.toarray())
kmeans = KMeans(n_clusters=k_number, random_state=0).fit(X)     
print()

##리스트로 된 군집화 결과를 kmeans_result에 넣는다.
kmeans_result = kmeans.labels_
print("::: K-Means result", ":" * 100)
print(kmeans_result)

##군집화가 완료된 기사끼리 들어갈 리스트
kmeans_group = [[0 for col in range(0)] for row in range(k_number)]
print()



##    
for i in range(len(kmeans_result)):
    kmeans_group[kmeans_result[i]].append(dflist[i])


    
for i in range(len(kmeans_group)):
    print(i,"그룹의 개수는",len(kmeans_group[i])) 


::: K-Means result ::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
[3 4 3 ... 3 3 4]

0 그룹의 개수는 28
1 그룹의 개수는 100
2 그룹의 개수는 108
3 그룹의 개수는 565
4 그룹의 개수는 204
5 그룹의 개수는 110
6 그룹의 개수는 58
7 그룹의 개수는 42
8 그룹의 개수는 69
9 그룹의 개수는 105
10 그룹의 개수는 44
11 그룹의 개수는 75
12 그룹의 개수는 209
13 그룹의 개수는 51


In [35]:
kmeans_group_url = [[0 for col in range(0)] for row in range(k_number)]
for i in range(len(kmeans_result)):
    kmeans_group_url[kmeans_result[i]].append(dfurl[i])

In [36]:
kmeans_group_company = [[0 for col in range(0)] for row in range(k_number)]
for i in range(len(kmeans_result)):
    kmeans_group_company[kmeans_result[i]].append(dfcompany[i])

In [42]:
kmeans_group_period = [[0 for col in range(0)] for row in range(k_number)]
for i in range(len(kmeans_result)):
    kmeans_group_period[kmeans_result[i]].append(dfperiod[i])

In [45]:
kmeans_group_title = [[0 for col in range(0)] for row in range(k_number)]
for i in range(len(kmeans_result)):
    kmeans_group_title[kmeans_result[i]].append(dftitle[i])

In [6]:
###======================countVectorizer 단계===========================
vectorize = CountVectorizer(
    tokenizer=tokenizer, 
    min_df=2    # min_df = 0.01 : 문서의 1% 미만으로 나타나는 단어 무시
                # min_df = 10 : 문서에 10개 미만으로 나타나는 단어 무시
                # max_df = 0.80 : 문서의 80% 이상에 나타나는 단어 무시
                # max_df = 10 : 10개 이상의 문서에 나타나는 단어 무시
)




countVectorizer_data = []

for i in range(len(kmeans_group)):
    temp = vectorize.fit_transform(kmeans_group[i])
    countVectorizer_data.append(temp.toarray())
    print("group",i,"countVectorizer data\n",countVectorizer_data[i])
    print()


group 0 countVectorizer data
 [[1 0 2 ... 0 0 0]
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

group 1 countVectorizer data
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

group 2 countVectorizer data
 [[0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

group 3 countVectorizer data
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

group 4 countVectorizer data
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

group 5 countVectorizer data
 [[0 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]

group 6 countVectorizer data
 [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [

In [46]:
###======================코사인 유사도 계산 단계===========================
checked = [[0 for col in range(0)] for row in range(k_number)]
similarity_simple_pair = [] 

for i in range(len(kmeans_group)):
    similarity_simple_pair.append(cosine_similarity(countVectorizer_data[i] , countVectorizer_data[i]))

# for i in range(len(kmeans_group)):    
#     print("group0 cosine similarity:\n",similarity_simple_pair[i])
#     print()
print("======================================================================") 

# print(similarity_simple_pair[0])


for h in range(len(similarity_simple_pair)):
    for i in range(len(similarity_simple_pair[h])):
        for j in range(len(similarity_simple_pair[h][i])):
            if(0.9 <  similarity_simple_pair[h][i][j] <= 0.99999):
                if(i not in checked[h]):
                    checked[h].append(i)
                    if(j not in checked[h]):
                        checked[h].append(j)

In [60]:
### 결과 출력하기 
## 유의사항 : dflist 에서 기사를 가져오는 것이 아니라 kmeans_group 내에서 가져와야함. (인덱스 뒤섞임 방지)

for h in range(len(kmeans_group)):
    print("::::::::::::",h,"번째 그룹 ::::::::::::::")
    for i in checked[h]:
        print(i,")", kmeans_group_company[h][i], kmeans_group_period[h][i])
        print("기사 제목: ",kmeans_group_title[h][i])
        print("기사 본문: ",kmeans_group[h][i].replace('\t', '').replace('\n', ''))
        print()
    print()
    

:::::::::::: 0 번째 그룹 ::::::::::::::

:::::::::::: 1 번째 그룹 ::::::::::::::
1 ) 한국경제 1996-03-01
기사 제목:  임금인상 33%가 타사 눈치/경총 95임금실태조사
기사 본문:  임금인상률을 결정할때 다른 기업의 임금수준과 인상률에 신경을 쓰는 눈치보기 성향이 확산되고 있고 임금협상 기간과 횟수도 늘어난 것으로 나타났다. 20일 한국경영자총협회가 작성한 "95년도 임금조정 실태조사 자료집"에 따르면 국내 2천86개 기업중 지난해 임금인상률을 결정할때 다른 기업의 임금수준 및 인상률을 가장 먼저 고려했다는 곳이 33.2%로 가장 많았다.다음은 지불능력 29.8%, 물가상승률 13.9%, 정부의 임금 가이드라인 10.8%,종업원의 생계비 9.3%등의 순이었다. 이같은 결과는 지불능력 30.1%, 다른기업의 임금수준 및 인상률 27.4%,중앙노사 임금합의 14.6%등으로 조사된 전년에 비해 임금인상률 결정에 눈치보기 성향이 확산되고 있는 것으로 분석됐다.<권영설 기자>

48 ) 한국일보 1996-03-01
기사 제목:  임금인상률 결정 「눈치보기」 확산/경총 작년 실태조사
기사 본문:  ◎“타기업수준 먼저 고려” 33.2%임금인상률을 결정할때 다른 기업의 임금수준과 인상률을 고려하는 경향이 심하며 임금협상 기간과 횟수도 늘어난 것으로 나타났다.20일 한국경영자총협회가 작성한 「95년도 임금조정 실태조사 자료집」에 따르면 국내 2,086개 기업중 지난해 임금인상률을 결정할 때 다른 기업의 임금수준 및 인상률을 가장 먼저 고려했다는 곳이 33.2%로 가장 많았다. 다음으로는 지불능력 29.8%, 물가 상승률 13.9%, 정부의 임금가이드라인 10.8%, 종업원의 생계비 9.3%, 기타 3.0% 등의 순이었다.이같은 결과는 지불능력 30.1%, 다른 기업의 임금수준 및 인상률 27.4%, 중앙노사 임금합의 14.6% 등으로 조사된 전년에 비해 임금인상률 결정에 눈치보기성향이 확산되고 있음을 보여주는 것이

In [31]:
for i in checked[1]:
    print(i, kmeans_group_url[1][i])

1 https://www.bigkinds.or.kr/news/newsDetailView.do?newsId=02100601.20160116020952314
48 https://www.bigkinds.or.kr/news/newsDetailView.do?newsId=01101101.19960321000001108
12 https://www.bigkinds.or.kr/news/newsDetailView.do?newsId=01601001.19960309154617026
15 https://www.bigkinds.or.kr/news/newsDetailView.do?newsId=01500701.20160409145434315
22 https://www.bigkinds.or.kr/news/newsDetailView.do?newsId=02100601.20160116015041785
23 https://www.bigkinds.or.kr/news/newsDetailView.do?newsId=01601101.19960304134112103
24 https://www.bigkinds.or.kr/news/newsDetailView.do?newsId=01500601.19960304003728001
27 https://www.bigkinds.or.kr/news/newsDetailView.do?newsId=02100311.19960327201924102
31 https://www.bigkinds.or.kr/news/newsDetailView.do?newsId=01300101.20160615054943904
28 https://www.bigkinds.or.kr/news/newsDetailView.do?newsId=01100611.20160420211755918
30 https://www.bigkinds.or.kr/news/newsDetailView.do?newsId=01101001.19960327000000201
32 https://www.bigkinds.or.kr/news/newsDetai

In [57]:
print(kmeans_group[0][0].replace('\t', '').replace('\n', ''))
# name = name.replace('\t', '').replace('\n', '')

◎노트북 핸드폰 등 첨단무기로 무장/삶의 설계외 재테크·세무상담도생활설계사가 여성 최고의 인기직종으로 부상하고 있다. 한때 「보험아줌마」로 불렸던 적도 있지만 더이상 아줌마들의 전유물이 아니다. 오히려 고학력 대졸 아가씨가 몰려들고 있다.불확실한 내일을 위한 삶의 설계자로서의 역할이 부각되고 나아가 개인이나 가정의 세무상담,재테크 등에 대한 조언까지 담당하는 재무설계사로서 그 영역을 넓혀가고 있다. 그래서 이들은 보험사의 꽃으로 불린다.연봉 수억원을 자랑하는 생활설계사도 이젠 낯설지 않다. 전문성과 젊음을 내세운 이들은 휴대용컴퓨터와 핸드폰 등 첨단 「무기」로 무장하고 직장사무실 곳곳을 파고들고 있다.생활설계사의 고학력 경향은 사회가 다양화하고 세분화되면서 단순히 상품에 대한 지식뿐 아니라 총체적인 분석력이 요구되기 때문이다. 삼성생명은 92년 대졸생활설계사 50명을 공개채용,고학력 바람을 일으켰고 이같은 추세는 보험업계 전반에 확산됐다. 지난해 교보생명이 실시한 대졸생활설계사 공개채용에는 1천여명이 몰려 인기를 실감케 했다.보험감독원에 따르면 94년 1억원이상 고액의 수입을 올린 생활설계사는 1백50여명에 이르며 3억원이상을 벌어들인 경우도 있었다.그러나 생활설계사는 여전히 「힘들고 어려운」직업이다. 생면부지의 사람을 만나 눈에 보이지 않은 상품을 팔야야 한다. 프로의식과 전문성을 갖춘 사람만이 성공할 수 있다. 이제 부업이 아닌 전업직종으로 자리매김하고 있는 것이다.〈박현동〉◎삼성화재 「천만인 운전자보험」/“자동차구입서 폐차까지 모든 문제 책임”『자동차는 현대인들에게 필수품으로 인식되고 보유대수도 확대되고 있지만 예기치 못한 불의의 자동차 사고로 당신의 인생은 수렁으로 빠져들 수 있습니다. 삼성화재의 「천만인 운전자보험」은 당신의 인생을 안전하게 보호해줄 것입니다』삼성화재 테헤란영업소 장혜정씨(27·여)는 자동차는 생활필수품이라며 「천만인 운전자보험」이 「인생의 보호자」라고 단언했다. 장씨는 『천만인 운전자보험은 언제 발생할지 모르는 사고로 항상 불안감에 